teegide importimine

Esimene rida impordib numpy mooduli, mida kasutatakse laialdaselt teaduslike arvutuste jaoks, eriti matemaatiliste operatsioonide jaoks. 

Teine rida impordib os mooduli, mis võimaldab teha tegevusi failisüsteemiga, näiteks lugeda ja kirjutada faile.

Järgmised read impordivad mitmeid klasside definitsioone Keras'i teegist. Need klassid hõlmavad mitmesuguseid kihte nagu Dense, Flatten, Reshape, Input, Conv2D, Conv2DTranspose, Dropout ja LeakyReLU.

In [ ]:
import numpy as np
import os
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Reshape, Input, Conv2D, Conv2DTranspose, Dropout, LeakyReLU
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

read defineerivad hüperparameetreid

„Epochs“ määrab, kui palju koolitus tsükleid tehakse,

„batch_size“ määrab kui suure partii kaupa andmeid korraga koolitatakse,

„steps_per_epoch“ määrab, kui palju koolitusetappe tehakse iga tsükli kohta,

„latent_dim“ määrab, mida kasutatakse võrgu sisendiks,

„img_shape“ määrab piltide kuju, 

„learning_rate“ ning „beta_1“ määravad „Adam“ optimeerija hüperparameetrid 

In [ ]:
# Define hyperparameters
epochs = 100
batch_size = 32
steps_per_epoch = 254  
latent_dim = 100
img_shape = (128, 128, 1)
learning_rate = 0.0002
beta_1 = 0.5

read laevad andmeid 

In [ ]:
# Load data
data = ImageDataGenerator(rescale=1./255).flow_from_directory(
    '/kaggle/input',
    target_size=(128, 128),
    batch_size=batch_size,
    color_mode='grayscale'

defineeritakse diskriminaator võrgu mudel

Mudeli struktuur koosneb viiest kihist. Sisendkiht võtab vastu pildid. Sellele järgneb 3 konvolutsioonikihti (Conv2D), mis aitavad tuvastada olulisi mustreid ja omadusi piltidel. Pärast seda on täielikult ühendatud kiht, mis teeb otsuse selle kohta, kas pilt on autentne või võlts. Viimane kiht väljastab väljundina tõenäosuse, et pilt on ehtne. 

In [ ]:
# Define discriminator model
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=4, strides=2, padding="same", input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))
    model.summary()
    img = Input(shape=img_shape)
    validity = model(img)
    return Model(img, validity)

defineeritakse generatiivse võrgu mudel

Mudeli struktuur koosneb viiest kihist. Sisendkiht võtab vastu „noise“ sisendi, mis on juhusliku müra vektori kujul. Sellele järgneb „Dense“ kiht ja veel 3 konvolutsioonikihti (Conv2DTranspose). Need kihid aitavad luua võrgule spetsiifilisi kujundeid ja mustreid.

In [ ]:
# Define generator model
def build_generator():
    model = Sequential()
    model.add(Dense(128 * 16 * 16, activation="relu", input_dim=latent_dim))
    model.add(Reshape((16, 16, 128)))
    
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))
    
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
    
    model.add(Conv2DTranspose(1, kernel_size=4, strides=2, padding="same", activation="tanh"))
    model.summary()
    noise = Input(shape=(latent_dim,))
    img = model(noise)
    return Model(noise, img)

See mudel koosneb generaatori ja diskriminaatori mudelitest

In [ ]:
# Define GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    gan_output = discriminator(generator(gan_input))
    gan = Model(gan_input, gan_output)
    gan.summary()
    return gan

Antud kood loob ja kompileerib kolm mudelit 

In [ ]:
# Build and compile discriminator model
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate, beta_1=beta_1), metrics=['accuracy'])

# Build and compile generator model
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate, beta_1=beta_1))

# Build GAN model
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate, beta_1=beta_1))

In [ ]:
# Define arrays to store loss values
d_losses = []
g_losses = []
import tensorflow as tf
tf.debugging.set_log_device_placement(True)

Treenimise protsessi koodi osa algab spetsifitseerides GPU (graafikaprotsessori) indeksiga 0 kasutamise TensorFlow'i meetodiga „tf.device“. Seejärel algab koolitustsükkel, kus igas epohhis väljastatakse hetke epohhi number ja epohhide koguarv. Igas epohhis koolitatakse diskriminaatorit ja generaatorit meetodiga „train_on_batch“.   

In [ ]:
with tf.device('GPU:0'):
    # Begin training loop
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for step in range(steps_per_epoch):

            # Train discriminator on real images
            real_images, _ = data.next()
            real_labels = np.ones((batch_size, 1))
            d_loss_real = discriminator.train_on_batch(real_images, real_labels)

            # Train discriminator on fake images
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_images = generator.predict(noise)
            fake_labels = np.zeros((batch_size, 1))
            d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)

            # Calculate discriminator loss and accuracy
            d_loss, d_acc = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            real_labels = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(noise, real_labels)

        # Print loss values
        print(f"Discriminator loss: {d_loss}, accuracy: {d_acc}")
        print(f"Generator loss: {g_loss}")

        # Save loss values for plotting
        d_losses.append(d_loss)
        g_losses.append(g_loss)

        # Generate example images
        if epoch % 10 == 0:
            noise = np.random.normal(0, 1, (1, latent_dim))
            generated_image = generator.predict(noise)
            plt.imshow(generated_image[0, :, :, 0], cmap='gray')
            plt.show()

Ridade 7-10 vahel treenitakse diskriminaatorit reaalsete piltidega. „real_images, _ = data.next()“ laeb järgmise partii reaalsete piltide andmeid ja „_“ tähistab teist väljundit, mida ei kasutata. 
„real_labels“ moodustab maatriksi, kus kõik elemendid on 1, mis tähendab, et diskriminaatorile antakse sisse reaalsed pildid ja eesmärk on nende tõelisust korrektselt klassifitseerida. 
„discriminator.train_on_batch(real_images, real_labels)“ meetod treenib diskriminaatorit antud partii andmetega ning tagastab kaod, mida kasutatakse hiljem diskriminaatori kadude ja täpsuse arvutamisel.

Ridadel 12-16 treenitakse diskriminaatorit võltsitud piltidega. Esmalt genereeritakse valik juhuslikke vektoreid „noise“. 
Seejärel moodustatakse võltsitud piltidele maatriks „fake_labels“, kus kõik elemendid on nullid. 
Meetod „discriminator.train_on_batch(fake_images, fake_labels)“ treenib diskriminaatorit võltsitud piltidega ja tagastab kaod, mida kasutatakse hiljem diskriminaatori kadude ja täpsuse arvutamisel.

Ridade 18-19 vahel arvutatakse diskriminaatori kaod ja täpsus, kasutades „np.add“ meetodit, et leid keskmistatud kaotus tõeliste ja võltsitud piltide vahel.
 
Seejärel ridade 22-24 vahel treenitakse generaatorit, kasutades juhuslikult genereeritud mürasignaale (mis vastavad võrgu sisendile) ning märgistades neid tõelistena (real_labels). See tähendab, et generaatoril on eesmärk luua pilt, mis sarnaneb reaalsele pildile, mis omakorda petab diskriminaatorit. Treeningu lõppedes tagastab meetod „g_loss“, mis kajastab generaatori kaotust antud partiiga treenides. 

Ridade 26-32 vahel väljastatakse ekraanile diskriminaatori ja generaatori kaod, mida kasutatakse mudeli hindamiseks. Seejärel lisatakse need kaod vastavatesse kaotuste listidesse (d_losses ja g_losses), mis on hiljem kasutatud kaotuste kuvamiseks graafikul. Iga 10 epohhi järel genereeritakse üks näidispilt, mille kuvab seejärel matplotlib. Selline lähenemine võimaldab mudeli treeningu ajal visualiseerida, kuidas generatiivne mudel areneb.


